In [22]:
from urllib.request import urlopen
import json
import pandas as pd

In [23]:
drivers = pd.read_csv('drivers.csv')
drivers

,driver_number,broadcast_name,full_name,name_acronym,team_name,team_colour,first_name,last_name,headshot_url,country_code,session_key,meeting_key
0,1,M VERSTAPPEN,Max VERSTAPPEN,VER,Red Bull Racing,3671C6,Max,Verstappen,https://www.formula1.com/content/dam/fom-websi...,NED,7768,1141
1,2,L SARGEANT,Logan SARGEANT,SAR,Williams,37BEDD,Logan,Sargeant,https://www.formula1.com/content/dam/fom-websi...,USA,7768,1141
2,4,L NORRIS,Lando NORRIS,NOR,McLaren,F58020,Lando,Norris,https://www.formula1.com/content/dam/fom-websi...,GBR,7768,1141
3,10,P GASLY,Pierre GASLY,GAS,Alpine,2293D1,Pierre,Gasly,https://www.formula1.com/content/dam/fom-websi...,FRA,7768,1141
4,11,S PEREZ,Sergio PEREZ,PER,Red Bull Racing,3671C6,Sergio,Perez,https://www.formula1.com/content/dam/fom-websi...,MEX,7768,1141
...,...,...,...,...,...,...,...,...,...,...,...,...
2031,44,L HAMILTON,Lewis HAMILTON,HAM,Mercedes,27F4D2,Lewis,Hamilton,https://media.formula1.com/d_driver_fallback_i...,GBR,9617,1247
2032,55,C SAINZ,Carlos SAINZ,SAI,Ferrari,E80020,Carlos,Sainz,https://media.formula1.com/d_driver_fallback_i...,ESP,9617,1247
2033,63,G RUSSELL,George RUSSELL,RUS,Mercedes,27F4D2,George,Russell,https://media.formula1.com/d_driver_fallback_i...,GBR,9617,1247
2034,77,V BOTTAS,Valtteri BOTTAS,BOT,Kick Sauber,52E252,Valtteri,Bottas,https://media.formula1.com/d_driver_fallback_i...,FIN,9617,1247


In [24]:
sessions = drivers.session_key.unique().tolist()

In [25]:
tyres = pd.DataFrame(columns=['meeting_key', 'session_key', 'stint_number', 'driver_number', 
                              'lap_start', 'lap_end', 'compound', 'tyre_age_at_start'])

In [26]:
from urllib.error import HTTPError
import time

for session in sessions:
    print('Downloading data for session', session,)
    
    for attempt in range(3):  # 3 deneme hakkı
        try:
            # API isteği gönder
            response = urlopen('https://api.openf1.org/v1/stints?session_key=' + str(session))

            data = json.loads(response.read().decode('utf-8'))
            # Veri boşsa döngüden çık
            if len(data) == 0:
                break
            
            for tyre in data:
                # JSON verisini normalize et ve tyres DataFrame'ine ekle
                tyre_data = pd.json_normalize(tyre)
                tyres = pd.concat([tyres, tyre_data], ignore_index=True)

            
            break  # Başarılı olduğunda döngüden çık
        except HTTPError as e:
            if e.code == 429:  # Hız sınırı hatası
                print("Rate limit reached. Waiting before retrying...")
                time.sleep(5)  # 5 saniye bekle
            else:
                print(f"HTTP Error: {e.code}. Skipping session {session}.")
                break
    else:
        print(f"Failed to fetch session {session} after 3 attempts.")


In [27]:
tyres

,meeting_key,session_key,stint_number,driver_number,lap_start,lap_end,compound,tyre_age_at_start
0,1141,7768,1,1,1,1,SOFT,0
1,1141,7768,1,4,1,1,SOFT,0
2,1141,7768,1,11,1,1,SOFT,0
3,1141,7768,1,81,1,1,SOFT,0
4,1141,7768,1,27,1,2,SOFT,0
...,...,...,...,...,...,...,...,...
7030,1247,9617,2,30,37,57,MEDIUM,0
7031,1247,9617,3,20,39,57,MEDIUM,0
7032,1247,9617,2,43,40,57,MEDIUM,0
7033,1247,9617,2,63,41,57,MEDIUM,0


In [28]:
tyres.to_csv('tyres.csv', index=False)